# Import libraries

In [ ]:
!pip install rdkit

In [ ]:
import rdkit, rdkit.Chem, rdkit.Chem.Draw
from rdkit.Chem import Descriptors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import colormaps
from rdkit.Chem import Descriptors
from pandas.plotting import scatter_matrix

# Read the database

In [ ]:
url_corso= 'https://raw.githubusercontent.com/stefano-bosio/CTF_ML_MD/main/Lipophilicity.csv'

In [ ]:
data = pd.read_csv(url_corso)

In [ ]:
data

In [ ]:
molecules=[rdkit.Chem.MolFromSmiles(smi) for smi in data['smiles']]

In [ ]:
molecules[10]

In [ ]:
subset=[x for x in molecules[:9]]

In [ ]:
img=rdkit.Chem.Draw.MolsToGridImage(subset,molsPerRow=3,subImgSize=(300,300))
img

In [ ]:
valid_mols=[x for x in molecules if x != None]

In [ ]:
len(molecules),len(valid_mols)

In [ ]:
features=pd.DataFrame()
for i,mol in enumerate(molecules):
    features.loc[i,'MolWt']=Descriptors.MolWt(mol)
    features.loc[i,'NumHAcceptors']=Descriptors.NumHAcceptors(mol)
    features.loc[i,'NumHDonors']=Descriptors.NumHDonors(mol)
    features.loc[i,'NumRotBonds']=Descriptors.NumRotatableBonds(mol)
    features.loc[i,'NumHeteroatoms']=Descriptors.NumHeteroatoms(mol)
    features.loc[i,'FractionCSP3']=Descriptors.FractionCSP3(mol)
    features.loc[i,'RingCount']=Descriptors.RingCount(mol)
    features.loc[i,'TPSA']=Descriptors.TPSA(mol)
    features.loc[i,'Stereocenters']=rdkit.Chem.rdMolDescriptors.CalcNumAtomStereoCenters(mol)
    features.loc[i,'Spiro']=rdkit.Chem.rdMolDescriptors.CalcNumSpiroAtoms(mol)



In [ ]:
features

# Scaling of variables

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
features_scaled=pd.DataFrame(MinMaxScaler().fit_transform(features), columns=features.columns)

In [ ]:
print(features.min(),"\n")
print(features.max(),"\n")
print(features_scaled.min(),"\n")
print(features_scaled.max(),"\n")

# Compute correlation

In [ ]:
correlations=features_scaled.corr()
correlations

In [ ]:
ticks=[x for x  in features_scaled.columns]

In [ ]:
ticks

In [ ]:
plt.figure(figsize=(8,8))
plt.matshow(np.array(correlations), fignum=0, cmap='RdBu')
plt.xticks(np.linspace(0,9,10),ticks,rotation= 90)
plt.yticks(np.linspace(0,9,10),ticks)
plt.colorbar()
plt.clim((-1,1))

In [ ]:
scatter_matrix(features_scaled, figsize=(20,20))
plt.show()

# Feature Selection

## 1. Univariate Linear Filtering



In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest, r_regression
import pandas as pd

In [ ]:
X = np.array(features_scaled)
Y = data['exp']
feat=features.columns

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
test=r_regression(X,Y)

In [ ]:
scores=pd.DataFrame(feat,columns=['feat'])
scores['scores']=test
scores.sort_values(by='scores', ascending=False, inplace=True)
print(scores)

## 2. Wrapping through Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
data=[]

for i in range(len(feat)):
  n_feat=len(feat)-i

  rfe = RFE(model, n_features_to_select=n_feat)   # seek for n_feat features
  fit = rfe.fit(X, Y)

  data.append((fit.ranking_))
  scores=pd.DataFrame(feat,columns=['feat'])
  scores['scores']=fit.ranking_
  scores.sort_values(by='scores',ascending=True,inplace=True)

  print(f"Model with {n_feat} features")
  print(scores)
  print('\n')

In [ ]:
rfe = RFECV(model)   # seek for best number of features
fit = rfe.fit(X, Y)

scores=pd.DataFrame(feat,columns=['feat'])
scores['scores']=fit.ranking_
scores.sort_values(by='scores',ascending=True,inplace=True)

print(f"Model with {fit.n_features_} features")
print(scores)

In [ ]:
fit.cv_results_['mean_test_score']

In [ ]:
plt.plot(range(1,len(feat)+1),fit.cv_results_['mean_test_score'],marker='o')
plt.errorbar(x=fit.cv_results_["n_features"], y=fit.cv_results_["mean_test_score"], yerr=fit.cv_results_["std_test_score"],c='Tab:Blue')

plt.xlabel("# Features")
plt.ylabel("CV score")

plt.grid()